sequentially split the trace

In [ ]:
import pandas as pd
import numpy as np


nums = ['050', '100', '200', '250', '300']
for num in nums:
    path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_pod_list_cpu' + num + '.csv'
    df = pd.read_csv(path)
    # Filter the dataframe
    running_df = df[df['pod_phase'] == 'Running']
    # running_df = running_df[running_df['qos'] == 'LS']
    running_df = running_df[running_df['num_gpu'] == 0]
    
    tenants = [1, 2, 3, 5, 10, 15, 20]
    for tenant in tenants:
        # Split the dataframe into parts
        dfs = np.array_split(running_df, tenant)
        # Save each part to a separate CSV file
        for i, df_part in enumerate(dfs, 1):
            out_file = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/fabing_cpu_{}_{}_{}.csv'.format(num, tenant, i)
            print('data/fabing_cpu_{}_{}_{}'.format(num, tenant, i))
            df_part.to_csv(out_file.format(i), index=False)


split the table

In [ ]:
# Filter the dataframe
running_df = df[df['pod_phase'] == 'Running']

# running_df = running_df[running_df['qos'] == 'LS']
running_df = running_df[running_df['num_gpu'] == 0]

# Save to a new CSV file
running_df.to_csv('/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/fabing_running_all.csv', index=False)

# Split the dataframe into 3 equal parts
dfs = np.array_split(running_df, 3)

# Save each part to a separate CSV file
for i, df_part in enumerate(dfs, 1):
    df_part.to_csv(f'/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/fabing_running_part{i}_all.csv', index=False)

Output the number of used nodes

In [ ]:
import pandas as pd

path = '/home/fabing/projects/kubernetes-scheduler-simulator/experiments/2023_1123/fabing_cpu_300_20_13/05-BestFit/1.3/42/analysis_allo.csv'
df = pd.read_csv(path)

df.columns = [x.split('-')[-1] for x in df.columns]
dfd = df.to_dict(orient="list")
print(max(df['used_nodes']))
total_gpu_num = df.total_gpus.values[0]
df['arrive_ratio'] = df.arrived_gpu_milli / total_gpu_num / 10
df['arrive_ratio'] = df['arrive_ratio'].apply(lambda x: round(x, 0))
df['alloc_ratio'] = df.used_gpu_milli / total_gpu_num / 10
df['alloc_ratio'] = df['alloc_ratio'].apply(lambda x: round(x, 2))

Further processing of analysis results

In [ ]:
import pandas as pd

# Load the dataframe from the CSV file
df = pd.read_csv('/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_allo_discrete.csv')
df.drop(columns=['sc_policy', 'tune', 'seed'], inplace=True)

# Group by 'workload' and calculate the mean of the other columns
df = df.groupby('workload').mean()
# Reset the index
df = df.reset_index()
# Extract the prefix of the 'workload' column
df['workload_prefix'] = df['workload'].str.rsplit('_', n=1).str[0]

# Calculate the average 'use_nodes' for each 'workload_prefix'
average_use_nodes = df.groupby('workload_prefix')['use_nodes'].sum()
average_use_nodes.to_csv('/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_allo_discrete_sum_use_nodes_uniform.csv')
df = pd.read_csv('/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_allo_discrete_sum_use_nodes_uniform.csv')
# Split the 'workload_prefix' into four parts
df[['prefix', 'cpu', 'first_number', 'second_number']] = df['workload_prefix'].str.split('_', n=3, expand=True)

# Convert the 'first_number' and 'second_number' columns to numeric types
df[['first_number', 'second_number']] = df[['first_number', 'second_number']].apply(pd.to_numeric)

# Group by 'first_number' and calculate the mean of 'use_nodes'
grouped_df = df.groupby('first_number')['use_nodes'].mean()

# Sort by 'second_number'
sorted_df = df.sort_values('first_number')

print(sorted_df)
sorted_df.to_csv('/home/fabing/projects/kubernetes-scheduler-simulator/experiments/analysis/analysis_results/analysis_allo_discrete_sum_use_nodes_uniform.csv')


filter the node config according to 'memory' and 'cpu'

In [ ]:
import pandas as pd

# Load the dataframe from the CSV file
path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_node_list_all_node.csv'
df = pd.read_csv(path)

# Remove columns ['sn', 'gpu', 'model']
df = df.drop(columns=['sn', 'gpu', 'model'])

# Remove duplicate rows
df = df.drop_duplicates()

# Save to a new CSV file
df.to_csv('/home/fabing/projects/kubernetes-scheduler-simulator/simplify_node_list_all_node.csv', index=False)
print(df)


Uniformly sample the trace and generate the corresponding table

In [ ]:
import pandas as pd


nums = ['050', '100', '200', '250', '300']
for num in nums:
    path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_pod_list_cpu' + num + '.csv'
    df = pd.read_csv(path)
    # Filter the dataframe
    running_df = df[df['pod_phase'] == 'Running']
    # running_df = running_df[running_df['qos'] == 'LS']
    running_df = running_df[running_df['num_gpu'] == 0]
    
    tenants = [1, 2, 3, 5, 10, 15, 20]
    for tenant in tenants:
        # Split the dataframe into parts
        dfs = [df.iloc[i::tenant].reset_index(drop=True) for i in range(tenant)]
        # Save each part to a separate CSV file
        for i, df_part in enumerate(dfs, 1):
            out_file = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/fabing_uniform_{}_{}_{}.csv'.format(num, tenant, i)
            print('data/fabing_uniform_{}_{}_{}'.format(num, tenant, i))
            df_part.to_csv(out_file.format(i), index=False)
            # print(df_part.head(2))


Add the price to the node list

In [3]:
import pandas as pd

# Load the dataframe from the CSV file
path = '/home/fabing/projects/kubernetes-scheduler-simulator/data/csv/openb_node_list_all_node.csv'
node_df = pd.read_csv(path)

# Load the price dataframe from the CSV file
path = '/home/fabing/projects/kubernetes-scheduler-simulator/node_list_price.csv'
price_df = pd.read_csv(path)
# Remove commas and convert to float
price_df['Aliyun cost (yuan/month)'] = price_df['Aliyun cost (yuan/month)'].replace(',', '', regex=True).astype(float)

result_df = pd.merge(node_df, price_df, on=['cpu_milli', 'memory_mib'], how='left')
result_df.to_csv('/home/fabing/projects/kubernetes-scheduler-simulator/node_list_all_node_price.csv', index=False)